# Step 1: Create a Recharge File for MODFLOW

## Assign a precip value to each model grid cell centroid

In [2]:
import geopandas as gpd
import os
import numpy as np
from osgeo import gdal
import pandas as pd

## Read in the model CSV from Texas Water Development Board 

URL: https://www.twdb.texas.gov/groundwater/models/gam/glfc_n/glfc_n.asp


## CSV is also available in this repository in GIS folder

In [3]:
df = pd.read_csv(os.path.join('..', 'GIS','ebfz_b_grid_poly082615.csv'))

## Determine number of rows and columns in the model
### nrow and ncol are the maximum number of row and col in model

In [4]:
nrow, ncol = df['ROW'].max(), df['COL'].max() 
nrow,ncol

(120, 120)

In [5]:
delr, delc = 1000, 500 # BUT should prob just read from .dis file

In [6]:
nlay = 1  

In [13]:
def get_raster_value(xcoords,ycoords,nrow,ncol,mfrows,mfcols,raster_path,name='name'):
    """
    :param xcoords:
    :param ycoords:
    :param nrow:
    :param ncol:
    :param mfrows:
    :param mfcols:
    :param raster_path:
    :param name:
    :return:

    Written by Ross Kushnereit
    INTERA, Inc.

    This function converts a raster to a NumPy array.  Then resizes it to match the model array.
    """
    dataset = gdal.Open(raster_path)
    band = dataset.GetRasterBand(1)
    cols, rows = dataset.RasterXSize, dataset.RasterYSize
    transform = dataset.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]
    data = np.array(band.ReadAsArray(0, 0, cols, rows))
    print(data.shape)
    array = np.ones((nrow,ncol)) * -12345
    for i in range(len(xcoords)):
        if (xcoords[i]>xOrigin) and (yOrigin > ycoords[i]):# and (xcoords[i]<(xOrigin+ncol*pixelWidth)) and (yOrigin+nrow*pixelHeight < ycoords[i]):
            try:
                col = abs(int((xcoords[i] - xOrigin) / pixelWidth))
                row = abs(int((yOrigin - ycoords[i]) / pixelHeight))
                r, c = int(mfrows[i]-1), int(mfcols[i]-1)
                # print(r,c, 'r,c')
                v = data[row, col]
                array[r, c] = v
            except:
                pass

    array[array<=-12345] = np.nan
    return array

##  The .TIF file comes from PRISM precipitation data.  The .TIF has been clipped to the model area to reduce the amount of data and to make the script run faster

In [14]:
raster_path = os.path.join('..', 'GIS','annual_30_prism.tif')
print(raster_path)

../GIS/annual_30_prism.tif


In [17]:
array = get_raster_value(df['CentroidX'],df['CentroidY'],nrow,ncol,df['ROW'],df['COL'],raster_path=raster_path,name='prism')

(678, 1711)


## Finally, save the array we just created to a .txt file

In [18]:
np.savetxt(os.path.join('..', 'output', 'prism_mean_rch.txt'), array)